> **Importando a lib Pandas**

In [32]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import cross_val_score
from sklearn.svm import LinearSVC

> **Criando um DataFrame à partir do arquivo .csv**

In [6]:
df = pd.read_csv('situacao_do_cliente.csv')

> **Criando DataFrames para características e resultados**

In [7]:
X_df = df[['recencia', 'frequencia', 'semanas_de_inscricao']]
Y_df = df['situacao']

> **Criando Dummies das variáveis não numéricas**

> Dummies são usados para transformar categorias *string* em várias categorias *binárias*.

In [8]:
Xdummies_df = pd.get_dummies(X_df)
Ydummies_df = Y_df

> **Criando listas à partir dos DataFrames**

In [9]:
X = Xdummies_df.values
Y = Ydummies_df.values

> **Criando dados de treino** *(80%)* **, dados de teste** *(10%)* **e dados de validação** *(10%)*

In [86]:
tamanho_de_treino = int(0.8 * len(Y))
#tamanho_de_validacao = int(len(Y) -  tamanho_de_treino)

#0-800
treino_dados = X[:tamanho_de_treino]
treino_marcacoes = Y[:tamanho_de_treino]

#800-999
validacao_dados = X[tamanho_de_treino:]
validacao_marcacoes = Y[tamanho_de_treino:]

> **Criando função para rodar algortimos em k-fold**

In [87]:
def fit_and_predict(nome, modelo, treino_dados, treino_marcacoes):
    score = cross_val_score(modelo, treino_dados, treino_marcacoes, cv = 10)
    taxa_de_acerto = np.mean(score)
    print('Taxa de acerto do algoritmo {0}: {1}%'.format(nome, round(taxa_de_acerto,3)*100))
    return taxa_de_acerto

In [137]:
def teste_real(modelo, validacao_dados, validacao_marcacoes):
    resultado = modelo.predict(validacao_dados)

    acertos = resultado == validacao_marcacoes

    total_de_acertos = sum(acertos)
    total_de_elementos = len(validacao_marcacoes)

    taxa_de_acerto = 100.0 * total_de_acertos / total_de_elementos

    print("Taxa de acerto do vencedor entre os dois algoritmos no mundo real: {0}%".format(taxa_de_acerto))

In [138]:
resultados = {}

> Rodando algoritmos

In [139]:
from sklearn.ensemble import AdaBoostClassifier # Para usar AdaBoost
modelo_AdaBoostClassifier = AdaBoostClassifier()
resultado_AdaBoostClassifier = fit_and_predict('AdaBoostClassifier', modelo_AdaBoostClassifier, treino_dados, treino_marcacoes)
resultados['resultadoAdaBoost'] = resultado_AdaBoostClassifier

Taxa de acerto do algoritmo AdaBoostClassifier: 76.3%


In [140]:
from sklearn.naive_bayes import MultinomialNB # Para Naive Bayes
modelo_MultinomialNB = MultinomialNB()
resultado_MultinomialNB = fit_and_predict('MultinomialNB', modelo_MultinomialNB, treino_dados, treino_marcacoes)
resultados['resultadoMultinomialNB'] = resultado_MultinomialNB

Taxa de acerto do algoritmo MultinomialNB: 83.0%


In [141]:
from sklearn.multiclass import OneVsRestClassifier
modelo = OneVsRestClassifier(LinearSVC(random_state=0))
resultadoOneVsRest = fit_and_predict('OneVsRest', modelo, treino_dados, treino_marcacoes)
resultados['resultadoOneVsRest'] = resultadoOneVsRest

Taxa de acerto do algoritmo OneVsRest: 92.30000000000001%


In [142]:
from sklearn.multiclass import OneVsOneClassifier 
modelo_OneVsOneClassifier = OneVsOneClassifier(LinearSVC(random_state=0))
resultado_OneVsOneClassifier = fit_and_predict('OneVsOneClassifier', modelo_OneVsOneClassifier, treino_dados, treino_marcacoes)
resultados['resultadoOneVsOne'] = resultado_OneVsOneClassifier

Taxa de acerto do algoritmo OneVsOneClassifier: 99.4%


In [143]:
resultados

{'resultadoAdaBoost': 0.76294719642242859,
 'resultadoMultinomialNB': 0.82997721018231851,
 'resultadoOneVsOne': 0.99444444444444446,
 'resultadoOneVsRest': 0.9231381148950808}

> Testando algoritmo vencedor com dados de validacao

In [145]:
modelo_OneVsOneClassifier.fit(treino_dados, treino_marcacoes)

OneVsOneClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0),
          n_jobs=1)

In [146]:
teste_real(modelo_OneVsOneClassifier, validacao_dados, validacao_marcacoes)

Taxa de acerto do vencedor entre os dois algoritmos no mundo real: 100.0%
